In [28]:
import pandas as pd
import numpy as np
import json
import tweepy
from pymongo import MongoClient
from pymongo import ASCENDING, DESCENDING
import itertools
import logging

In [4]:
with open('keys.json', 'r') as fopen:
    keys = json.load(fopen)
print(keys)

{'consumer_key': '9YT5tBUGaglpD2vv2XGkkQ4mK', 'consumer_secret': 'RHVOGy9NxNIocn3q9kn4te1QC1FmTOsdx8GO3AW0Nye2f3gSJb', 'access_token': '431391451-pjMSEw7oXMHQhzgSd653daKIGFCDhpWhQZIXnD9g', 'access_token_secret': 'qdZxnBdNNn74UvSi2MLQU9sblD6dBM9uWqe08SqtiV08c'}


In [5]:
auth = tweepy.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Scrap tweet by hashtag

In [ ]:
data_hashtag = tweepy.Cursor(api.search, q='#covid19 -filter:retweets -filter:replies', result_type='recent', lang='id').items()

try:
    for tweet_source in data_hashtag:
        if tweet_source != []:
            client = MongoClient('127.0.0.1', 27017)
            db = client.SNA
            collection = db.tweet_source
            collection.insert(tweet_source._json)
except Exception as e:
    print(e)

Rate limit reached. Sleeping for: 483
<ipython-input-4-10f0f6ac5ba6>:9: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  collection.insert(tweet_source._json)
Rate limit reached. Sleeping for: 788


# Scrap Retweet

In [6]:
client = MongoClient('127.0.0.1', 27017)
db = client.SNA
collection = db.tweet_source
data = collection.find({}).sort([('retweet_count', DESCENDING)])
hashtag_timeline = list(data)

In [7]:
for tweet_id in hashtag_timeline[0:100]:
    try:
        data = api.retweets(tweet_id.get('id'))
        for result in data:
            if result != []:
                client = MongoClient('127.0.0.1', 27017)
                db = client.SNA
                collection = db.retweet
                collection.insert(result._json)
    except Exception as e:
        print(e)
        continue

<ipython-input-7-0a9cd7aac24a>:9: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  collection.insert(result._json)
Rate limit reached. Sleeping for: 854


# Scrap Tweet Replies

In [7]:
for timeline in hashtag_timeline[0:100]:
    try:
        replies = tweepy.Cursor(api.search, q='to:{}'.format(timeline.get('user').get('screen_name')),
                                since_id=timeline.get('user').get('id'),
                                tweet_mode='extended').items(100)
        for reply in replies:
            try:
                    client = MongoClient('127.0.0.1', 27017)
                    db = client.SNA
                    collection = db.reply
                    collection.insert(reply._json)

#             except tweepy.RateLimitError as e:
#                 logging.error("Twitter api rate limit reached".format(e))
#                 continue

#             except tweepy.TweepError as e:
#                 logging.error("Tweepy error occured:{}".format(e))
#                 break

#             except StopIteration:
#                 break

            except:
                pass
    except:
        pass

<ipython-input-7-4390424f6cfc>:11: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  collection.insert(reply._json)
Rate limit reached. Sleeping for: 775
Rate limit reached. Sleeping for: 784
Rate limit reached. Sleeping for: 792


In [29]:
for x in hashtag_timeline[0:100]:
    name = x.get('user').get('screen_name')
    tweet_id = x.get('id')
    try:
        for tweet in tweepy.Cursor(api.search, q='to:' + name, result_type='recent', since_id=tweet_id).items(1000):
            if hasattr(tweet, 'in_reply_to_status_id_str'):
                if tweet.in_reply_to_status_id_str is not None:
                    client = MongoClient('127.0.0.1', 27017)
                    db = client.SNA
                    collection = db.reply
                    collection.insert(tweet._json)
    except Exception as e:
        print(e)
        continue

<ipython-input-29-726a776ee7ab>:11: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  collection.insert(tweet._json)


KeyboardInterrupt: 

In [26]:
tweet._json

{'created_at': 'Mon Apr 19 15:39:20 +0000 2021',
 'id': 1384169754387042308,
 'id_str': '1384169754387042308',
 'text': '@KKMPutrajaya MEMANG TAKDE HATI PERUT KE?! KERJA NAK TINDAS RAKYAT JE! MAKAN LA DUIT SAMAN KAMI RAKYAT MISKIN NI SA… https://t.co/1UFTFUf6A7',
 'truncated': True,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'KKMPutrajaya',
    'name': 'KKMalaysia🇲🇾',
    'id': 531041640,
    'id_str': '531041640',
    'indices': [0, 13]}],
  'urls': [{'url': 'https://t.co/1UFTFUf6A7',
    'expanded_url': 'https://twitter.com/i/web/status/1384169754387042308',
    'display_url': 'twitter.com/i/web/status/1…',
    'indices': [117, 140]}]},
 'metadata': {'iso_language_code': 'in', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': 531041640,
 'in_reply_to_user_id_str': '531041640',
 'i